In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Celiac_Disease"
cohort = "GSE113469"

# Input paths
in_trait_dir = "../../input/GEO/Celiac_Disease"
in_cohort_dir = "../../input/GEO/Celiac_Disease/GSE113469"

# Output paths
out_data_file = "../../output/preprocess/Celiac_Disease/GSE113469.csv"
out_gene_data_file = "../../output/preprocess/Celiac_Disease/gene_data/GSE113469.csv"
out_clinical_data_file = "../../output/preprocess/Celiac_Disease/clinical_data/GSE113469.csv"
json_path = "../../output/preprocess/Celiac_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the series title and overall design, this dataset contains PBMC gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: we can see 'disease state' in key 0 with values 'Healthy Control' and 'Celiac Disease'
trait_row = 0

# For age: we can see 'age' in key 1 with multiple values
age_row = 1  

# For gender: not available in the sample characteristics dictionary
gender_row = None  

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait data to binary (0: control, 1: celiac disease)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if it exists
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "healthy control" in value.lower():
        return 0
    elif "celiac disease" in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    """Convert age data to continuous numeric values"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if it exists
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender data to binary (0: female, 1: male)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if it exists
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    value = value.lower()
    if "female" in value or "f" == value:
        return 0
    elif "male" in value or "m" == value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Sample characteristics from the previous step
    sample_chars_dict = {
        0: ['disease state: Healthy Control', 'disease state: Celiac Disease'],
        1: ['age: 30', 'age: 27', 'age: 31', 'age: 26', 'age: 28', 'age: 32', 'age: 41', 'age: 34', 
            'age: 25', 'age: 42', 'age: 29', 'age: 21', 'age: 44', 'age: 56', 'age: 50', 'age: 51', 'age: 37'],
        2: ['cell type: PBMCs']
    }
    
    # Need to reconstruct clinical_data in the expected format for geo_select_clinical_features
    # Each row is a feature and columns are samples
    # First, let's determine the maximum sample size
    max_samples = max(len(values) for values in sample_chars_dict.values())
    
    # Create a properly structured DataFrame
    clinical_data = pd.DataFrame(index=range(len(sample_chars_dict)), columns=range(max_samples))
    
    # Fill the DataFrame with available data
    for row_idx, values in sample_chars_dict.items():
        for col_idx, value in enumerate(values):
            clinical_data.loc[row_idx, col_idx] = value
    
    # Now call geo_select_clinical_features with this properly structured data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# The identifiers starting with "ILMN_" are Illumina probe IDs, not human gene symbols
# These are microarray probe identifiers from Illumina BeadArray platform
# They need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns for probe IDs and gene symbols in the gene annotation
# Based on the preview, 'ID' contains the probe identifiers (same format as gene expression data index: ILMN_XXXXXXX)
# 'Symbol' contains the gene symbols
probe_id_col = 'ID'
gene_symbol_col = 'Symbol'

# 2. Create the gene mapping dataframe using the identified columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col=probe_id_col, gene_col=gene_symbol_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few genes after mapping:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was saved in Step 2
clinical_data_path = out_clinical_data_file
if os.path.exists(clinical_data_path):
    clinical_data = pd.read_csv(clinical_data_path)
    print(f"Clinical data loaded from {clinical_data_path}, shape: {clinical_data.shape}")
    
    # Load the original clinical data from the matrix file to ensure we have proper structure
    # Since we need to match the sample IDs between gene data and clinical data
    _, original_clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Transpose gene data so samples are in rows
    gene_data_t = gene_data.transpose()
    print(f"Transposed gene data shape: {gene_data_t.shape}")
    
    # Create a linked dataframe
    linked_data = pd.DataFrame()
    
    # Add trait data (first column of clinical_data)
    if clinical_data.shape[1] >= 1:
        # The clinical data might need to be transposed to match gene data sample ordering
        if len(clinical_data) == 2 and clinical_data.shape[1] == 17:  # Based on preview in Step 2
            # Transpose clinical data so samples are rows
            clinical_data_t = clinical_data.transpose()
            
            # Rename columns appropriately
            if clinical_data_t.shape[1] == 2:
                clinical_data_t.columns = [trait, 'Age']
                
                # Create DataFrame with clinical data columns first
                linked_data = clinical_data_t.copy()
                
                # Add gene expression data
                for gene in gene_data.index:
                    if gene in linked_data.columns:
                        # Avoid duplicate column names
                        continue
                    linked_data[gene] = gene_data_t[gene].values
                
                print(f"Linked data shape: {linked_data.shape}")
                
                # 3. Handle missing values
                linked_data = handle_missing_values(linked_data, trait)
                print(f"Shape after handling missing values: {linked_data.shape}")
                
                # 4. Determine if trait and demographic features are biased
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # 5. Conduct final quality validation
                is_usable = validate_and_save_cohort_info(
                    is_final=True, 
                    cohort=cohort, 
                    info_path=json_path, 
                    is_gene_available=True, 
                    is_trait_available=True,
                    is_biased=is_biased,
                    df=linked_data,
                    note="This dataset contains gene expression data from celiac disease patients on gluten-free diet versus controls."
                )
                
                # 6. Save linked data if usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Dataset deemed not usable based on quality validation, linked data not saved.")
            else:
                print("Clinical data structure doesn't match expected format (2 columns).")
                is_usable = validate_and_save_cohort_info(
                    is_final=True, 
                    cohort=cohort, 
                    info_path=json_path, 
                    is_gene_available=True, 
                    is_trait_available=False,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="Clinical data structure doesn't match expected format."
                )
        else:
            print("Clinical data structure doesn't match expected format.")
            is_usable = validate_and_save_cohort_info(
                is_final=True, 
                cohort=cohort, 
                info_path=json_path, 
                is_gene_available=True, 
                is_trait_available=False,
                is_biased=True,
                df=pd.DataFrame(),
                note="Clinical data structure doesn't match expected format."
            )
    else:
        print("Clinical data is empty or missing trait column, cannot create linked data.")
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,
            is_biased=True,
            df=pd.DataFrame(),
            note="Clinical data loaded but appears to be empty or missing trait information."
        )
else:
    print(f"Clinical data file {clinical_data_path} not found, cannot create linked data.")
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note="Clinical data file not found."
    )

print(f"Dataset usability status: {'Usable' if is_usable else 'Not usable'}")